**Домашнее задание**

In [5]:
import pandas as pd

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [6]:
# Импортирую данные о рейтингах фильмов

ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
# Испортирую данные о названиях фильмов

movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [54]:
# Группирую таблицу с рейтингами по ID фильмов и рассчитываю среднюю оценку для каждого фильма.
# Объединяю две таблицы в одну, добавив рейтинг названия в таблицу с рейтингами фильмов.

ratings_group = ratings.groupby('movieId').mean()[['rating']].reset_index()
joined = ratings_group.merge(movies, on='movieId', how='left')

# Не придумала, как поменять местами колонки и убрать лишние, поэтому еще раз использую группировку.
joined = joined.groupby(['movieId', 'title']).mean()[['rating']].reset_index().sort_values(by='rating', ascending=False)
joined.head()

,movieId,title,rating
9065,163949,The Beatles: Eight Days a Week - The Touring Y...,5.0
7297,71180,Padre padrone (1977),5.0
6629,51471,Amazing Grace (2006),5.0
6662,52617,Woman on the Beach (Haebyeonui yeoin) (2006),5.0
6704,53887,O Lucky Man! (1973),5.0


In [55]:
# Функция для расчета класса фильма, в зависимости от его оценки

def ratings_class(row):
    if row['rating'] < 2:
        return 'Низкий рейтинг'
    elif (row['rating'] >= 2) and (row['rating'] < 4.5):
        return 'Средний рейтинг'
    elif 4.5 <= row['rating'] <= 5:
        return 'Высокий рейтинг'
    else:
        return 'Ошибка!'
    

joined['class'] = joined.apply(ratings_class, axis = 1)
joined.head()

,movieId,title,rating,class
9065,163949,The Beatles: Eight Days a Week - The Touring Y...,5.0,Высокий рейтинг
7297,71180,Padre padrone (1977),5.0,Высокий рейтинг
6629,51471,Amazing Grace (2006),5.0,Высокий рейтинг
6662,52617,Woman on the Beach (Haebyeonui yeoin) (2006),5.0,Высокий рейтинг
6704,53887,O Lucky Man! (1973),5.0,Высокий рейтинг


In [56]:
# Для проверки, считаю сколько фильмов с разными классами рейтинга

joined.groupby(['class']).count()[['rating']]

,rating
class,
Высокий рейтинг,668
Низкий рейтинг,596
Средний рейтинг,7802


**Задание 2**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [62]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [61]:
# Испортирую таблицу

keywords = pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [82]:
# Функция для поиска названия города в словаре

def geo_check(keyword):
    geo_data_dict = geo_data
    for k, v in geo_data_dict.items():
        if any(word in keyword for word in v):
            return k

# Функция для проверки таблицы с ключами и присвоения региона ключу        
def region_(row):
    if geo_check(row['keyword']):
        return geo_check(row['keyword'])
    else:
        return 'undefined'

        
keywords['region'] = keywords.apply(region_, axis = 1)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [85]:
# Проверка работы функций
print(keywords['region'].value_counts())

filtered_keywords = keywords [ ~(keywords['region'].str.contains('undefined'))]
filtered_keywords.head()

# Можно ли короче как-то сделать решение? Я не придумала лучше сама.

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64


,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


**Задание 3 (бонусное)**

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга